Scene-Text Detection

In [2]:
# Loading all necessary libraries and modules
import os
import cv2
import csv
import math
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [3]:
def get_cwd(join_ch):
    dir = os.getcwd().strip().replace("\\", join_ch)
    dir = dir.split(join_ch)[:-1]
    dir = join_ch.join(dir)+join_ch
    return dir

In [15]:
main_dir = os.getcwd().strip()
dataset_name = '/voc'
dataset_dir = os.path.join(main_dir, 'dataset'+dataset_name).strip().replace("\\", "/")
dataset_dir

'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc'

In [21]:
#dataset annotation csv preparation
file_map_dict = {}
annot_map_dict = {}

data_dirs = ['train', 'test']

for fname in os.listdir(dataset_dir):
    path = os.path.join(dataset_dir, fname).strip().replace("\\", "/")
    all_files = os.listdir(path)
    if fname in data_dirs:
        img_files = [os.path.join(path, f).strip().replace("\\", "/") for f in all_files if f[-3:] == 'jpg']
        file_map_dict[fname] = [os.path.relpath(f).strip().replace("\\", "/") for f in img_files]
    if fname == 'Annotations':
        annot_files = [os.path.join(path, f).strip().replace("\\", "/") for f in all_files if f[-3:] == 'xml']
        annot_map_dict[fname] = [os.path.relpath(f).strip().replace("\\", "/") for f in annot_files]


In [24]:
len(annot_map_dict['Annotations'])
len(file_map_dict['train'])

400

In [44]:
train_annots = list()
test_annots = list()

annot_path = path = os.path.join(dataset_dir, 'Annotations').strip().replace("\\", "/")

for ind, each in enumerate([fs.split("/")[-1][:-4] for fs in file_map_dict['train']] ):
    if each in [annot.split("/")[-1][:-4] for annot in annot_map_dict['Annotations']]:
        train_annots.append(os.path.join(annot_path, each+'.xml').replace("\\", "/"))

for ind, each in enumerate([fs.split("/")[-1][:-4] for fs in file_map_dict['test']] ):
    if each in [annot.split("/")[-1][:-4] for annot in annot_map_dict['Annotations']]:
        test_annots.append(os.path.join(annot_path, each+'.xml').replace("\\", "/"))

In [45]:
train_annots
test_annots

['e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009129.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009131.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009138.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009141.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009144.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009147.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009148.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/Git-Repo/Code/text_classification/dataset/voc/Annotations/009155.xml',
 'e:/Gustovalley/PhD Projects/Text segmentation/

In [66]:
import xml.etree.ElementTree as ET

xmlfile = train_annots[0]
tree = ET.parse(xmlfile)
root = tree.getroot()

size_elem = root.find('size')
file_w = size_elem.find('width').text
file_h = size_elem.find('height').text

name_elem = root.find('filename')
filename = name_elem.text

bbox_elem = root.find('object').find('bndbox')
xmin = bbox_elem.find('xmin').text
ymin = bbox_elem.find('ymin').text
xmax = bbox_elem.find('xmax').text
ymax = bbox_elem.find('ymax').text

print(xmin,ymin, xmax, ymax)


AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'txt'

In [59]:
annot_data = {}
for fname in data_dirs:
    annot_data[fname] = {}
    temp = annot_data[fname]
    for files in annot_map_dict[fname]:
        with open(files, 'r') as f:
            temp[os.path.split(files)[1]] = f.readlines()

In [234]:
def get_rot_rect_coords(center:tuple, width: int, height: int, angle: float):
    # TOP RIGHT VERTEX:
    x1 = int(center[0] + ((width / 2) * math.cos(angle)) - ((height / 2) * math.sin(angle)))
    y1 = int(center[1] + ((width / 2) * math.sin(angle)) + ((height / 2) * math.cos(angle)))

    # TOP LEFT VERTEX:
    x2 = int(center[0] - ((width / 2) * math.cos(angle)) - ((height / 2) * math.sin(angle)))
    y2 = int(center[1] - ((width / 2) * math.sin(angle)) + ((height / 2) * math.cos(angle)))

    # BOTTOM LEFT VERTEX:
    x3 = int(center[0] - ((width / 2) * math.cos(angle)) + ((height / 2) * math.sin(angle)))
    y3 = int(center[1] - ((width / 2) * math.sin(angle)) - ((height / 2) * math.cos(angle)))

    # BOTTOM RIGHT VERTEX:
    x4 = int(center[0] + ((width / 2) * math.cos(angle)) + ((height / 2) * math.sin(angle)))
    y4 = int(center[1] + ((width / 2) * math.sin(angle)) - ((height / 2) * math.cos(angle)))

    return ((x1, y1), (x2, y2), (x3, y3), (x4, y4))

In [375]:
def rect_corners(center:tuple, x: int, y: int, angle: float):
    # cx, cy - center of square coordinates
    # x, y - coordinates of a corner point of the square
    # theta is the angle of rotation

    # translate point to origin
    tempX = x - center[0]
    tempY = y - center[1]

    # now apply rotation
    rotatedX = tempX*np.cos(angle) - tempY*np.sin(angle)
    rotatedY = tempX*np.sin(angle) + tempY*np.cos(angle)

    # translate back
    x = rotatedX + center[0]
    y = rotatedY + center[1]

    return (int(x),int(y))

In [393]:
whole_data = []

sample = {}
keys = ['filename','xmin','ymin','width','height','angle', 'center']

for fname in data_dirs:
    for k,v in annot_data[fname].items():
        for annots in v:
            main_dat = annots.replace('\n','').split(" ")[2:]
            sample['filename'] = k
            sample['xmin'] = int(main_dat[0])
            sample['ymin'] = int(main_dat[1])
            sample['width'] = int(main_dat[2])
            sample['height'] = int(main_dat[3])
            sample['angle'] = float(main_dat[4]) / math.pi * 180
            sample['center'] = (sample['xmin']+sample['width']/2, sample['ymin']+sample['height']/2)
            sample['coords'] = get_rot_rect_coords(sample['center'], sample['width'], sample['height'], sample['angle'])

            whole_data.append(sample)
            sample = {}

In [394]:
bbox_whole_data = []
bbox_final_data = []

for data in whole_data:
    points = cv2.boxPoints((data['center'], (data['width'], data['height']), data['angle']))
    bbox = np.int0(points)
    bbox_whole_data.append((data['filename'], bbox))
    bbox_final_data.append((data['filename'], data['coords']))

In [409]:
bbox_whole_data[0][1]

array([[751, 966],
       [746, 861],
       [793, 858],
       [798, 963]], dtype=int64)

In [396]:
bbox_final_data[0]

('IMG_0030.gt', ((770, 855), (814, 872), (774, 969), (730, 952)))

In [441]:
for ind, fpaths in enumerate(file_map_dict['train']):
    img = cv2.imread(fpaths)
    fnames = os.path.split(annot_map_dict['train'][ind])[1]
    for name, arr in bbox_whole_data:
        if fnames == name:
            cv2.line(img, tuple(arr[1]), tuple(arr[2]), (255,0,0), 15)
            cv2.line(img, tuple(arr[1]), tuple(arr[0]), (255,0,0), 15)
            cv2.line(img, tuple(arr[0]), tuple(arr[3]), (255,0,0), 15)
            cv2.line(img, tuple(arr[2]), tuple(arr[3]), (255,0,0), 15)
            cv2.drawContours(img, [arr], 0, (0,255,0), 2)
    img = cv2.resize(img, (900, 600))
    cv2.imshow("image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    if ind == 10:
        break

In [482]:
to_csv = [[name]+[str(val) for _, val in enumerate(arr.flatten())] for name, arr in bbox_whole_data]
to_csv = [['name','pt1','pt2','pt3','pt4','pt5','pt6','pt7','pt8']]+to_csv
to_csv


[['name', 'pt1', 'pt2', 'pt3', 'pt4', 'pt5', 'pt6', 'pt7', 'pt8'],
 ['IMG_0030.gt', '751', '966', '746', '861', '793', '858', '798', '963'],
 ['IMG_0030.gt', '728', '963', '727', '919', '743', '918', '744', '962'],
 ['IMG_0063.gt', '827', '653', '834', '582', '1074', '606', '1067', '677'],
 ['IMG_0064.gt', '626', '653', '613', '614', '863', '535', '876', '574'],
 ['IMG_0064.gt', '702', '728', '674', '634', '1272', '453', '1300', '547'],
 ['IMG_0064.gt', '732', '783', '717', '734', '1300', '557', '1315', '606'],
 ['IMG_0081.gt', '721', '687', '705', '524', '857', '508', '873', '671'],
 ['IMG_0081.gt', '645', '851', '639', '772', '991', '743', '997', '822'],
 ['IMG_0081.gt', '676', '915', '672', '849', '984', '826', '988', '892'],
 ['IMG_0155.gt', '299', '437', '274', '299', '794', '204', '819', '342'],
 ['IMG_0155.gt', '581', '521', '574', '482', '994', '410', '1001', '449'],
 ['IMG_0183.gt', '246', '413', '228', '321', '857', '195', '875', '287'],
 ['IMG_0451.gt', '646', '894', '624', 

In [483]:
with open('annotation.csv', 'w', encoding='utf8', newline='') as f:
    for row in to_csv:
        for x in row:
            f.write(x + ',')
        f.write('\n')

In [512]:
def rename_func():
    voc_path = os.path.join(dataset_dir, "voc/test").replace("\\", "/")
    files = [os.path.join(voc_path, f[:-11]+'.jpg').replace("\\", "/") for f in os.listdir(voc_path)]
    voc_files = [os.path.join(voc_path, f).replace("\\", "/") for f in os.listdir(voc_path)]
    new_files = [os.rename(old, new) for old, new in zip(voc_files, files)]